In [1]:
import torch
import random
import numpy as np
from functools import partial

In [ ]:
# pip install torch torch_geometric torch-scatter pandas scikit-learn wandb
# python -m pip install gurobipy

In [2]:
from temp.data.gen import parallel_generate_problem, parallel_generate_solutions
from temp.data.problem import setcover
from temp.data.info import ModelInfo
from temp.data.dataset import ModelGraphDataset
from temp.data.aug import parallel_augment_info


from temp.solver.utils import solve_inst


In [22]:
m = setcover(256, 256, 0.1)

In [23]:
m.setParam("NoRelHeurTime", 5)
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 256 rows, 256 columns and 6464 nonzeros
Model fingerprint: 0x99232920
Variable types: 0 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 281.0000000
Presolve time: 0.01s
Presolved: 256 rows, 256 columns, 6464 nonzeros
Variable types: 0 continuous, 256 integer (256 binary)

Root relaxation: objective 8.578826e+01, 354 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   85.78826    0   90  281.00000   85.78826  69.5%

In [3]:
PRE_TRAIN_DIR = "temp/pre_train"
TRAIN_DIR = "temp/train"
VALID_DIR = "temp/valid"

In [4]:
# parallel_generate_problem(setcover, PRE_TRAIN_DIR, 100, 10)
# parallel_generate_solutions(PRE_TRAIN_DIR, 10)

parallel_generate_problem(setcover, TRAIN_DIR, 5000, 10)
parallel_generate_solutions(TRAIN_DIR, 10)

# parallel_generate_problem(setcover, VALID_DIR, 100, 10)
# parallel_generate_solutions(VALID_DIR, 10)

Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Read LP format model from file temp/train/7_83.lp
Reading time = 0.00 seconds
: 100 rows, 200 columns, 1958 nonzeros
Read LP format model from file temp/train/4_131.lp
Reading time = 0.00 seconds
: 100 rows, 200 columns, 1982 nonzeros
Read LP format model from file temp/train/9_46.lp
Reading time = 

In [5]:
m = setcover()
vals = solve_inst(m)
info = ModelInfo.from_model(m)
info.var_info.sols = np.array([[m.objVal] + vals])

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 100 rows, 200 columns and 1958 nonzeros
Model fingerprint: 0xee762c29
Variable types: 0 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 226.0000000
Presolve time: 0.00s
Presolved: 100 rows, 200 columns, 1958 nonzeros
Variable types: 0 continuous, 200 integer (200 binary)

Root relaxation: objective 7.333071e+01, 227 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [6]:
from temp.data.dataset import info_to_data
d = info_to_data(info)
aug = partial(parallel_augment_info, n=100)

In [7]:
pre_train_d = ModelGraphDataset(PRE_TRAIN_DIR, augment=parallel_augment_info)
train_d = ModelGraphDataset(TRAIN_DIR, augment=parallel_augment_info)
valid_d = ModelGraphDataset(VALID_DIR)

data = pre_train_d[0][1]
var_feature_size = data.var_node_features.size(-1)
con_feature_size = data.con_node_features.size(-1) 

In [8]:
import pandas as pd
CFG = pd.read_excel("temp/trained_models/setcover_model_configs.xlsx", index_col=0).loc[0].to_dict()
CFG["num_epochs"] = 5
CFG["num_layers"] = 4
CFG["hidden"] = 8

In [9]:
from temp.model.utils import get_model
from temp.model.trainer import train

In [10]:
model_name, model, criterion, optimizer, scheduler = get_model(".", var_feature_size, con_feature_size, n_batches=1, **CFG)

In [11]:
scheduler.total_steps = 10000

In [12]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed()
    np.random.seed(worker_seed)
    random.seed(worker_seed)

from torch_geometric.loader import DataLoader
pretrain_loader = DataLoader(pre_train_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))
train_loader = DataLoader(train_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))
val_loader = DataLoader(valid_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))

In [13]:
scheduler.total_steps = 10000

In [14]:
train(model_name, model, criterion, optimizer, scheduler, pretrain_loader, train_loader, val_loader, CFG, False, "./")

>> Training starts on the current device cpu
>> Pretraining for prenorm...


  0%|          | 0/138 [00:00<?, ?it/s]


>> Epoch 1 ----------------------------------------------------------------------------------------------------
Training... 0


100%|██████████| 1375/1375 [03:03<00:00,  7.51it/s]


train_loss                0.564197
train_acc                 0.864560
train_f1                  0.675954
train_precision           0.729288
train_recall              0.629889
train_evidence_succ       3.263105
train_evidence_fail       2.168765
train_uncertainty_succ    0.418042
train_uncertainty_fail    0.505072
train_true_bias           0.224267
train_pred_bias           0.193700
train_soft_pred_bias      0.244598
train_bias_error          0.078373
train_lr                  0.000345
dtype: float32
Validating...


...

100%|██████████| 1375/1375 [02:57<00:00,  7.77it/s]


train_loss                 0.199452
train_acc                  0.952663
train_f1                   0.889983
train_precision            0.929426
train_recall               0.853752
train_evidence_succ       23.034817
train_evidence_fail        4.953551
train_uncertainty_succ     0.126342
train_uncertainty_fail     0.430183
train_true_bias            0.224267
train_pred_bias            0.206007
train_soft_pred_bias       0.208233
train_bias_error           0.018294
train_lr                   0.000165
dtype: float32
Validating...


100%|██████████| 13/13 [00:00<00:00, 19.12it/s]


val_loss                 0.243093
val_acc                  0.937100
val_f1                   0.450175
val_precision            0.551392
val_recall               0.380354
val_evidence_succ       25.408981
val_evidence_fail        9.241115
val_uncertainty_succ     0.115889
val_uncertainty_fail     0.399694
val_true_bias            0.067596
val_pred_bias            0.046058
val_soft_pred_bias       0.048444
val_bias_error           0.021538
val_lr                   0.000165
dtype: float32


MIPGNN